# Ex03 Aggregations

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect("../data/checking-logs.sqlite")
cur = con.cursor()

In [3]:
schema = pd.read_sql("PRAGMA table_info(test);", con)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


In [4]:
test_10 = pd.read_sql("SELECT * FROM test LIMIT 10", con)
test_10

,uid,labname,first_commit_ts,first_view_ts
0,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


In [5]:
df_min = pd.read_sql("""
            SELECT
                uid,
                MIN((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as min_diff
            FROM test t JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            """
            ,con)
df_min

,uid,min_diff
0,user_30,-202


In [6]:
df_max = pd.read_sql("""
            SELECT
                uid,
                MAX((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as max_diff
            FROM test t JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            """
            ,con)
df_max

,uid,max_diff
0,user_25,-2


In [7]:
df_avg = pd.read_sql("""
            SELECT
                AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as avg_diff
            FROM test t JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            """
            ,con)
df_avg


,avg_diff
0,-89.125


In [8]:
views_diff = pd.read_sql(""" 
            SELECT 
                t.uid,
                AVG((CAST(strftime('%s', first_commit_ts) AS INT) - deadlines) / 3600) as avg_diff,
                COUNT (*) AS pageviews
            FROM test t LEFT JOIN deadlines d ON d.labs = t.labname
            WHERE t.labname != 'project1'
            GROUP BY uid
            """,
            con)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,5
1,user_10,-74.800000,5
2,user_14,-159.000000,3
3,user_17,-61.600000,5
4,user_18,-5.666667,3
5,user_19,-98.750000,4
6,user_21,-95.500000,4
7,user_25,-92.600000,5
8,user_28,-86.400000,5
9,user_3,-105.400000,5


In [9]:
# Можно сделать вывод, что корреляции между средней дельтой и количеством просмотров страницы нет
views_diff.corr(numeric_only=True)

,avg_diff,pageviews
avg_diff,1.000000,0.117685
pageviews,0.117685,1.000000


In [10]:
con.close()